<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/DPT/DPT_inference_notebook_(depth_estimation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Define model

In [ ]:
from transformers import DPTFeatureExtractor, DPTForDepthEstimation

feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large")
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large")

## Prepare image

In [ ]:
from PIL import Image
import requests

# url = './data/original/2024-05-07_16-07-07_original.png'
url = '../../data/out/1.png'
image = Image.open(url)
image

In [3]:
pixel_values = feature_extractor(image, return_tensors="pt").pixel_values

## Forward pass

In [4]:
import torch

with torch.no_grad():
  outputs = model(pixel_values)
  predicted_depth = outputs.predicted_depth

## Visualize

In [ ]:
import numpy as np

# interpolate to original size
prediction = torch.nn.functional.interpolate(
                    predicted_depth.unsqueeze(1),
                    size=image.size[::-1],
                    mode="bicubic",
                    align_corners=False,
              ).squeeze()
output = prediction.cpu().numpy()
formatted = (output * 255 / np.max(output)).astype('uint8')
depth = Image.fromarray(formatted)
depth

In [6]:
depth.save("../../data/depth/1.png")